# compute basic statistics on llc4320 native grid

In [1]:
import numpy as np
import xarray as xr

from matplotlib import pyplot as plt
%matplotlib inline

import mitequinox.utils as ut
import mitequinox.plot as pl

In [2]:
if False:
    from dask.distributed import Client, LocalCluster
    from dask_jobqueue import PBSCluster
    cluster = PBSCluster(walltime='06:00:00')
    w = cluster.scale(jobs=20)
    client = Client(cluster)
else:
    from dask.distributed import Client, LocalCluster
    #
    cluster = LocalCluster()
    client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:35738 Dashboard: /user/pontea/proxy/8787/status,Cluster Workers: 4 Cores: 16 Memory: 64.42 GB


In [3]:
cluster

---
# compute simple diagnostics


In [4]:
# same but over all variables and faces
V = ["Eta", "SSU", "SSV", "SST", "SSS"]
#V = V+["KPPhbl", "qsoceTAUX", "oceTAUY"]

In [10]:
def compute_store(v, ds, diag, op):
    #i, j = ut.get_ij_dims(ds)
    diag_zarr = diag+"_"+v
    if not ut.is_diagnostic(diag_zarr):
        out, tmp_zarr = ut.custom_distribute(ds, op, 
                                             tmp_dir=ut.scratch+"tmp/", 
                                             face=1, ) # **{j: 1080}
        ut.store_diagnostic(diag_zarr, out, overwrite=True)
        print("{} {} stored".format(v, diag))
    else:
        print("{} {} already exists".format(v, diag))

In [ ]:
diags = {"mean": lambda x: x.mean("time"),
         "mean_square": lambda x: (x**2).mean("time"),
        }
        
for v in V:
    ds = ut.load_data(v)
    for diag, op in diags.items():
        compute_store(v, ds, diag, op)

Eta mean already exists
data stored in /work/ALT/swot/aval/syn/diags/square_mean_Eta.zarr
Eta square_mean stored
data stored in /work/ALT/swot/aval/syn/diags/mean_SSU.zarr
SSU mean stored
data stored in /work/ALT/swot/aval/syn/diags/square_mean_SSU.zarr
SSU square_mean stored
data stored in /work/ALT/swot/aval/syn/diags/mean_SSV.zarr
SSV mean stored


In [ ]:
cluster.close()

---

# plots ...

In [5]:
# load diagnostics
V = ["SSU", "SSV",]
D = ["mean", "mean_square"]

ds = xr.merge([ut.load_diagnostic(d+"_"+v).rename({v:d+"_"+v}) 
               for v in V for d in D])
#ds = ut.load_diagnostic ...

---

In [ ]:
client.restart()

In [8]:
cluster.close()